In [1]:
pip install chromedriver-autoinstaller

Note: you may need to restart the kernel to use updated packages.


In [2]:
from tqdm import tqdm
import pandas as pd
from sqlalchemy import create_engine
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from selenium.common.exceptions import TimeoutException
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import re
import sqlalchemy
from urllib.parse import quote
from selenium.common.exceptions import TimeoutException, WebDriverException

In [3]:
column_names = [
    'Index', 
    'Company_ID', 
    'Company_Name', 
    'Location', 
    'Status', 
    'Website'
]

# Read the CSV with custom headers
df = pd.read_csv(
    r"C:\Users\LTIM_10693340\Desktop\details.csv", 
    delimiter='\t', 
    header=None, 
    names=column_names, 
    on_bad_lines='skip'
)

# Replace the URLs
df['Website'] = df['Website'].str.replace(
    r'/company/', '/company-directors/', regex=False
)

C:\Users\LTIM_10693340\AppData\Local\Temp\ipykernel_15768\2971628115.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [5]:
# # import time
# # import logging
# # import chromedriver_autoinstaller
# # import undetected_chromedriver as uc
# # from selenium.webdriver.chrome.service import Service
# # from selenium.webdriver.common.by import By
# # from selenium.webdriver.support.ui import WebDriverWait
# # from selenium.webdriver.support import expected_conditions as EC
# # from selenium.common.exceptions import WebDriverException, TimeoutException
 
# # Configure logging

# logging.basicConfig(
#    level=logging.INFO,
#    format='%(asctime)s - %(levelname)s - %(message)s',
#    handlers=[

#       logging.StreamHandler(),

#        logging.FileHandler('cloudflare_bypass.log', mode='w')
#    ]
# )
 

 
 
 
# def create_driver():

#     # Enforce the specific ChromeDriver version that matches your Chrome browser version 13    chromedriver_autoinstaller.install("130.0.6723.117" 
#     # Set Chrome option
#     options = uc.ChromeOptions(
#     options.add_argument("--no-sandbox"
#     options.add_argument("--disable-dev-shm-usage"
#     options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.6723.117 Safari/537.36"
#     options.add_argument("--disable-blink-features=AutomationControlled"
#     # Initialize undetected ChromeDriver without version update
#     driver = uc.Chrome(options=options, version_main=130
#     return driver

In [ ]:
import time
import logging
import random
import re
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from urllib.parse import quote
import chromedriver_autoinstaller
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException, NoSuchElementException

# # Configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(levelname)s - %(message)s',
#     handlers=[
#         logging.StreamHandler(),
#         logging.FileHandler('cloudflare_bypass.log', mode='w')
#     ]
# )

# Configure logging
logging.basicConfig(
   level=logging.INFO,
   format='%(asctime)s - %(levelname)s - %(message)s',
   handlers=[

      logging.StreamHandler(),
       logging.FileHandler('cloudflare_bypass.log', mode='w')
   ]
)
 

# Define your database engine
engine = create_engine('mssql+pyodbc://SAL_USER01:%s@172.16.22.25:1433/SAL_DB?driver=ODBC+Driver+17+for+SQL+Server' % quote('Sal@123'))

# Log errors to a text file
def log_error(error_message):
    with open('error_log.txt', 'a') as f:
        f.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - {error_message}\n")

# def create_driver():
#     options = uc.ChromeOptions()
#     options.add_argument("--no-sandbox")
#     options.add_argument("--disable-dev-shm-usage")
#     options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.6723.92 Safari/537.36")
#     options.add_argument("--disable-blink-features=AutomationControlled")
#     return uc.Chrome(options=options)

def create_driver():
    # Enforce the specific ChromeDriver version that matches your Chrome browser version 13    chromedriver_autoinstaller.install("130.0.6723.117" 
    chromedriver_autoinstaller.install("131.0.6778.70")
    # Set Chrome option
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")

    options.add_argument("--disable-dev-shm-usage")

    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.6723.117 Safari/537.36")

    options.add_argument("--disable-blink-features=AutomationControlled")

    # Initialize undetected ChromeDriver without version update
    driver = uc.Chrome(options=options, version_main=131)
    return driver

# Retry logic for scraping failures
def safe_get(driver, url, retries=3, delay=5):
    for attempt in range(retries):
        try:
            driver.get(url)
            return True
        except (WebDriverException, TimeoutException) as e:
            log_error(f"Error loading {url} on attempt {attempt + 1}: {e}")
            time.sleep(delay)  # Delay before retrying
    return False

# Function to log in to Zaubacorp
def login_to_zaubacorp(driver, username, password):
    try:
        driver.get('https://www.zaubacorp.com/company-directors/AM-DAILY-SERVICES-OPC-PRIVATE-LIMITED/U74999MH2021OPC368405')
        print(f"Page title: {driver.title}")
        
        sign_in_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.LINK_TEXT, "Login")))
        sign_in_button.click()
        
        captcha_div = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//div[@class='form-type-textfield form-item-captcha-response form-item form-group']")))
        captcha_text = captcha_div.text
        print(f"Captcha text: {captcha_text}")
        
        captcha_answer = solve_captcha(captcha_text)
        if captcha_answer is None:
            log_error("Failed to solve captcha.")
            return False
        
        print(f"Captcha answer: {captcha_answer}")
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@name='name']"))).send_keys(username)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@name='pass']"))).send_keys(password)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@name='captcha_response']"))).send_keys(str(captcha_answer))
        
        login_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='edit-submit']")))
        login_button.click()
        print("Login form submitted.")
        return True
    except Exception as e:
        log_error(f"Error during login process: {e}")
        return False

def solve_captcha(captcha_text):
    try:
        captcha_question = re.search(r'(\d+ [\+\-\*/] \d+)', captcha_text)
        if captcha_question:
            expression = captcha_question.group()
            result = eval(expression)
            return result
    except Exception as e:
        log_error(f"Error solving captcha: {e}")
    return None

def scrape_directors(driver, start_index, end_index):
    director_data = []
    error_logs = []

    for url in df['Website'][start_index:end_index]:
        try:
            time.sleep(1)
            driver.get(url)
            url_parts = url.split('/')
            company_name = url_parts[-2]
            cin = url_parts[-1]
            past_directors_section = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, "//h3[contains(text(), 'Past Directors')]"))
            )
            director_details = past_directors_section.find_element(By.XPATH, "./following-sibling::table")
            rows = director_details.find_elements(By.TAG_NAME, "tr")
            for row in rows[1:]:
                columns = row.find_elements(By.TAG_NAME, "td")
                din = columns[0].text if len(columns) > 0 else None
                name = columns[1].text if len(columns) > 1 else None
                designation = columns[2].text if len(columns) > 2 else None
                start_date = columns[3].text if len(columns) > 3 else None
                end_date = columns[4].text if len(columns) > 4 else None
                director_data.append({
                    'DIN': din,
                    'Name': name,
                    'Designation': designation,
                    'Start Date': start_date,
                    'End Date': end_date,
                    'CIN': cin,
                    'Company_Name': company_name
                })
        except (WebDriverException, TimeoutException) as e:
            log_error(f"Error occurred for {url}: {e}")
            error_logs.append(f"Error for {url}: {e}")
            continue

    return director_data, error_logs

# Main loop to process in chunks of 500
def main():
    driver = create_driver()
    if not login_to_zaubacorp(driver, 'dummy3', 'Sal@2021'):
        driver.quit()
        return
    
    start_index = 1635002      
    total_urls = len(df['Website'])
    chunk_size = 1000
    chunk_limit = 65
    chunk_count = 0

    for start in range(start_index, total_urls, chunk_size):
        end = min(start + chunk_size, total_urls)
        print(f"Processing URLs from {start} to {end}")
        chunk_start_time = time.time()
        
        director_data_chunk, error_logs_chunk = scrape_directors(driver, start, end)
        chunk_duration = time.time() - chunk_start_time
        print(f"Chunk processed in: {time.strftime('%H:%M:%S', time.gmtime(chunk_duration))}")
        
        final = pd.DataFrame(director_data_chunk)
        if not final.empty:
            records_inserted = final.shape[0]
            distinct_cin_count = final['CIN'].nunique()
            final.to_sql('zauba_past_director', engine, if_exists='append', index=False)
            print(f'Total Records Inserted: {records_inserted}')
            print(f'Total Distinct CIN Inserted: {distinct_cin_count}')
        else:
            print('No data to insert for this chunk.')
        
        for log in error_logs_chunk:
            log_error(log)
        
        chunk_count += 1
        if chunk_count >= chunk_limit:
            print("Reached chunk limit. Stopping.")
            break
    
    driver.quit()

if __name__ == '__main__':
    main()


2024-12-09 18:13:40,172 - INFO - Chromedriver is already installed.
2024-12-09 18:13:50,761 - INFO - patching driver executable C:\Users\LTIM_10693340\appdata\roaming\undetected_chromedriver\undetected_chromedriver.exe


Page title: Just a moment...
Captcha text: Math question * 4 + 1 =
Captcha answer: 5
Login form submitted.
Processing URLs from 1635002 to 1636002
Chunk processed in: 00:30:58
Total Records Inserted: 1231
Total Distinct CIN Inserted: 992
Processing URLs from 1636002 to 1637002
Chunk processed in: 00:31:50
Total Records Inserted: 1187
Total Distinct CIN Inserted: 990
Processing URLs from 1637002 to 1638002
Chunk processed in: 00:35:28
Total Records Inserted: 1206
Total Distinct CIN Inserted: 992
Processing URLs from 1638002 to 1639002
Chunk processed in: 00:30:17
Total Records Inserted: 1216
Total Distinct CIN Inserted: 992
Processing URLs from 1639002 to 1640002
Chunk processed in: 00:32:37
Total Records Inserted: 1266
Total Distinct CIN Inserted: 988
Processing URLs from 1640002 to 1641002
Chunk processed in: 00:32:02
Total Records Inserted: 1250
Total Distinct CIN Inserted: 991
Processing URLs from 1641002 to 1642002
Chunk processed in: 00:29:15
Total Records Inserted: 1234
Total Dis

In [ ]:
pip show undetected_chromedriver